# Google Earth Engine

The data collected by Earth observing satellites has revolutionised the way we monitor the planet.

Every day new images are acquired that help document changes in ocean currents, global atmospheric circulation and the productivity of plants.

Google invented a tool called Google Earth Engine that combines satellite imagery with planetary-scale analysis capabilities, making it possible for scientists, researchers and developers to detect changes, map trends and quantify differences on the Earth's surface. Google Earth Engine [https://earthengine.google.com] is a free Google product supported by the Google Earth Outreach programme.

When using Google Earth Engine, researchers perform all their analysis on data hosted on Google's servers, and only view or download the results they need. The raw data never has to be housed on their own computers, nor do their local machines need to do any of the processing.

Explore GEE data catalog: https://developers.google.com/earth-engine/datasets

Timelapse feature: https://earthengine.google.com/timelapse/

Need more help: https://developers.google.com/earth-engine/tutorials/tutorials

End-to-End Google Earth Engine (Full Course): https://courses.spatialthoughts.com/end-to-end-gee.html

## Contents
* Visualizing feature collections
* Time Series Analysis

In [1]:
%matplotlib inline

### Installation Google Earth Engine API and geemap

Install the Earth Engine Python API and geemap. The geemap Python package is built upon the ipyleaflet and folium packages and implements several methods for interacting with Earth Engine data layers, such as Map.addLayer(), Map.setCenter(), and Map.centerObject(). The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its dependencies, including earthengine-api, folium, and ipyleaflet.

In [ ]:
!conda install -c conda-forge geemap

In [ ]:
!conda install -c conda-forge earthengine-api

You will be asked to confirm the installation of the API and its dependencies. After confirming, conda will download and install the dependencies. If all goes well, you will now have a conda environment called 'ee' with all the requirements for accessing the API, as well as the earthengine command line tool.

In [ ]:
#earthengine authenticate

In [ ]:
import ee
import geopandas as gpd
import subprocess
from IPython.display import Image
import ee, datetime
import pandas as pd
from pylab import *
import seaborn as sns
from matplotlib.pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose
import geemap

# Initialize the Earth Engine module.
ee.Initialize()

ee.Initialize() can take time to load

In [ ]:
print(ee.__version__)

In [ ]:
# Test Google Earth Engine Connected and working
# Print metadata for a DEM dataset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

### 1. Visualizing Feature Collection

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4)

In [ ]:
# Add Earth Engine dataset
# Load a FeatureCollection from a table dataset: 'RESOLVE' ecoregions.
ecoregions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')

# Display as default and with a custom color.
Map.addLayer(ecoregions, {}, 'default display')
Map.addLayer(ecoregions, {'color': 'FF0000'}, 'colored')


Map.addLayer(ecoregions.draw(**{'color': '006600', 'strokeWidth': 5}), {}, 'drawn')


# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()

# Paint all the polygon edges with the same number and 'width', display.
outline = empty.paint(**{
  'featureCollection': ecoregions,
  'color': 1,
  'width': 3
})
Map.addLayer(outline, {'palette': 'FF0000'}, 'edges')


# Paint the edges with different colors, display.
outlines = empty.paint(**{
  'featureCollection': ecoregions,
  'color': 'BIOME_NUM',
  'width': 4
})
palette = ['FF0000', '00FF00', '0000FF']
Map.addLayer(outlines, {'palette': palette, 'max': 14}, 'different color edges')


# Paint the edges with different colors and 'width's.
outlines = empty.paint(**{
  'featureCollection': ecoregions,
  'color': 'BIOME_NUM',
  'width': 'NNH'
})
Map.addLayer(outlines, {'palette': palette, 'max': 14}, 'different color, width edges')


# Paint the interior of the polygons with different colors.
fills = empty.paint(**{
  'featureCollection': ecoregions,
  'color': 'BIOME_NUM',
    })
Map.addLayer(fills, {'palette': palette, 'max': 14}, 'colored fills')


# Paint both the fill and the edges.
filledOutlines = empty.paint(ecoregions, 'BIOME_NUM').paint(ecoregions, 0, 2)
Map.addLayer(filledOutlines, {'palette': ['000000'] + palette, 'max': 14}, 'edges and fills')

In [ ]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

In [ ]:
# Add Earth Engine dataset
# Use an elevation dataset and terrain functions to create
# a custom visualization of topography.

# Load a global elevation image.
elev = ee.Image('USGS/GMTED2010')

# Zoom to an area of interest.
Map.setCenter(-121.069, 50.709, 6)

# Add the elevation to the map.
Map.addLayer(elev, {}, 'elev')

# Use the terrain algorithms to compute a hillshade with 8-bit values.
shade = ee.Terrain.hillshade(elev)
Map.addLayer(shade, {}, 'hillshade', False)

# Create a "sea" variable to be used for cartographic purposes
sea = elev.lte(0)
Map.addLayer(sea.mask(sea), {'palette':'000022'}, 'sea', False)

# Create a custom elevation palette from hex strings.
elevationPalette = ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff']
# Use these visualization parameters, customized by location.
visParams = {'min': 1, 'max': 3000, 'palette': elevationPalette}

# Create a mosaic of the sea and the elevation data
visualized = ee.ImageCollection([
  # Mask the elevation to get only land
  elev.mask(sea.Not()).visualize(**visParams),
  # Use the sea mask directly to display sea.
  sea.mask(sea).visualize(**{'palette':'000022'})
]).mosaic()

# Note that the visualization image doesn't require visualization parameters.
Map.addLayer(visualized, {}, 'elev palette', False)

# Convert the visualized elevation to HSV, first converting to [0, 1] data.
hsv = visualized.divide(255).rgbToHsv()
# Select only the hue and saturation bands.
hs = hsv.select(0, 1)
# Convert the hillshade to [0, 1] data, as expected by the HSV algorithm.
v = shade.divide(255)
# Create a visualization image by converting back to RGB from HSV.
# Note the cast to byte in order to export the image correctly.
rgb = hs.addBands(v).hsvToRgb().multiply(255).byte()
Map.addLayer(rgb, {}, 'styled')

In [ ]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

## 2. Proba-V Time-Series Analysis
Basic Time-Series Analysis using Proba-V NDVI (Normalized Difference Vegetation Index) imagery.
Read more about Proba-V: https://directory.eoportal.org/web/eoportal/satellite-missions/p/proba-v

#### Load Proba-V image collection and point geometry
Selected Location of point is from the Proba-V Footprint X18Y02 in Luxembourg, Europe. 

In [ ]:
# Set start and end date
startTime = datetime.datetime(2015, 1, 1)
endTime = datetime.datetime(2017, 12, 31)

# Create image collection
collection = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M').filterDate(startTime, endTime)
# Create point in Luxembourg (Proba-V Footprint: X18Y02)
point = {'type':'Point', 'coordinates':[6.134136, 49.612485]};

### Retrieve information, reshape and calculate NDVI
Retrieving information from point geometry with a buffer of 500m over image collection. Reshaping data and calculating NDVI from **RED** and **NIR** band.

In [ ]:
info = collection.getRegion(point,500).getInfo()

In [ ]:
# Reshape image collection 
header = info[0]
data = array(info[1:])

iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]

# List of used image bands
band_list = ['RED',u'NIR']

iBands = [header.index(b) for b in band_list]
yData = data[0:,iBands].astype(np.float)

# Calculate NDVI
red = yData[:,0]
nir = yData[:,1]
ndvi = (nir - red) / (nir + red)

### Reshape NDVI array into Pandas Dataframe

In [ ]:
df = pd.DataFrame(data=ndvi, index=list(range(len(ndvi))), columns=['NDVI'])
df = df.interpolate()
df['Date'] = pd.Series(time, index=df.index)
df = df.set_index(df.Date)
df.index = pd.to_datetime(df.index)
df['NDVI']=df['NDVI'].fillna(0)

#### Obtain statistical information over all elements of the Time-Series

In [ ]:
df.info()

In [ ]:
df.describe()

### Visualize Proba-V NDVI Time Series

In [ ]:
sns.set(rc={'figure.figsize':(15, 6)})
df['NDVI'].plot(linewidth=0.5);

### Seasonal Decomposition

In [ ]:
sd=seasonal_decompose(df['NDVI'], model='additive', freq=352)

sd.seasonal.plot()
sd.trend.plot()
sd.resid.plot()
plt.legend(['Seasonality', 'Trend', 'Residuals'])

In [ ]:
# Plot
plt.rcParams.update({'figure.figsize': (16,12)})
sd.plot().suptitle('ETS Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

### Resample data
Resampling to weekly level and calculate the percentage change over one year.

In [ ]:
df_monthly=df.resample('W').mean()
df_monthly['pct_change'] = df_monthly.pct_change(52)
df_monthly['pct_change']['2016':].plot()
plt.title('52 Weeks Percentage Change')